<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine/CNNmodel_v1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Builf train and evaluate
# Save model for offline scoring on test
# Model name: CCP_ICD10_Searchengine/CNNmodel_v1.0.ipynb
# References
#https://github.com/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments/Model-2-keras_lstmConv.ipynb
#https://github.com/njainds/NLP/blob/master/kaggle/QIQC/22nd%20Place%20Solution%20-%206%20models%20%2B%20POS%20tagging.ipynb
#https://github.com/Cheneng/DPCNN/blob/master/model/DPCNN.py


In [69]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
from fastai.text import *
import html
import sklearn
import torch.tensor as T
from sklearn.model_selection import train_test_split

#from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D
from keras import optimizers
import os
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

In [72]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
embedding_matrix.npy  mispell_dict.npy	test_X.npy  train_X.npy  word_index.npy
icd_dict.npy	      model_data.csv	test_y.npy  train_y.npy


In [0]:
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/")
x_train = np.load("train_X.npy")
x_test = np.load("test_X.npy")
y_train = np.load("train_y.npy")
y_test = np.load("test_y.npy")
features = np.load("test_y.npy")
icd_dict = np.load("icd_dict.npy").item()
word_index = np.load("word_index.npy").item()
ext_lexvec_embeddings = np.load("embedding_matrix.npy")

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

            
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [0]:
inp = Input(shape=(maxlen,))
x   = Embedding(max_features, embed_size, weights=[ext_lexvec_embeddings], trainable=False)(inp)
x   = SpatialDropout1D(rate=0.2)(x)
x   = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
x   = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
x1  = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x1   = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x1)
x_max   = GlobalMaxPooling1D()(x)
x_avg   = GlobalAveragePooling1D()(x)
x1_max  = GlobalMaxPooling1D()(x1)
x1_avg  = GlobalAveragePooling1D()(x1)
c   = concatenate([x_max,x_avg,x1_max,x1_avg])
y   = Dense(64, activation='relu')(c)
y   = Dropout(0.2)(y)
out = Dense(6, activation='sigmoid')(y)
model=Model(inputs=inp,outputs=out)
model.compile(loss='binary_crossentropy', optimizer=(lr=1e-3), metrics=["accuracy"])
print(model.summary())

In [0]:
model.fit(X_train, Y_train, batch_size = 128, epochs = 6, validation_data = (X_valid, Y_valid), verbose = 1, callbacks = [ra_val, check_point, early_stop])


In [0]:
pred = model.predict(x_test, batch_size = 1024, verbose = 1)